# ReelSense: Explainable Movie Recommender

This notebook demonstrates the ReelSense recommender system pipeline, including:
1. Data Loading & Preprocessing
2. Model Training (SVD, Hybrid, Neural CF, LightGCN, SASRec)
3. Ensemble Creation
4. Diversity Optimization (MMR)
5. Explainability

In [ ]:
import sys
import os
import pandas as pd
import numpy as np

# Add src to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.data import DataLoader
from src.recommenders import PopularityRecommender, CollaborativeRecommender, SVDRecommender, ContentRecommender, HybridRecommender, NeuralCFRecommender, DiversityRecommender, LightGCNRecommender, EnsembleRecommender, SASRecRecommender
from src.evaluation import precision_at_k, recall_at_k, ndcg_at_k, coverage
from src.explainability import Explainer

## 1. Load and Split Data

In [ ]:
data_path = "../ml-latest-small"
loader = DataLoader(data_path)
loader.load_data()
loader.preprocess()

train_df, test_df = loader.get_train_test_split(method='leave_last_n', n=1)
print(f"Train: {len(train_df)}, Test: {len(test_df)}")

## 2. Train Models (Including SOTA Transformers)

In [ ]:
all_items = loader.movies['movieId'].unique()

models = {
    "Popularity": PopularityRecommender(),
    "SVD": SVDRecommender(n_components=20),
    "Hybrid (SVD+Content)": HybridRecommender(SVDRecommender(n_components=20), ContentRecommender(), alpha=0.3),
    "Neural CF (NeuMF)": NeuralCFRecommender(embedding_dim=32, n_epochs=10),
    "LightGCN (SOTA)": LightGCNRecommender(n_epochs=50),
    "SASRec (Transformer)": SASRecRecommender(n_epochs=10, embedding_dim=64, n_heads=2),
    "Diversity Optimized (MMR)": DiversityRecommender(HybridRecommender(SVDRecommender(n_components=20), ContentRecommender(), alpha=0.3), lambda_param=0.6)
}

for name, model in models.items():
    print(f"Training {name}...")
    if "MMR" in name:
         model.fit(train_df, loader.movies, tags_df=loader.tags)
    elif "Hybrid" in name:
        model.fit(train_df, loader.movies, loader.tags)
    else:
        model.fit(train_df)

# Create Ensemble
print("Creating Ensemble...")
models["Ensemble (Trinity)"] = EnsembleRecommender({
    "SVD": models["SVD"],
    "LightGCN": models["LightGCN (SOTA)"],
    "SASRec": models["SASRec (Transformer)"]
}, weights={"SVD": 0.3, "LightGCN": 0.3, "SASRec": 0.4})

## 3. Evaluation

In [ ]:
from tqdm import tqdm

def evaluate(model, test_df, k=10):
    precisions, recs = [], []
    test_users = test_df['userId'].unique()
    
    for uid in tqdm(test_users):
        truth = test_df[test_df['userId'] == uid]['movieId'].tolist()
        rec_items = model.recommend(uid, n=k)
        if not rec_items: rec_items = []
        recs.append(rec_items)
        precisions.append(precision_at_k(rec_items, truth, k))
        
    return np.mean(precisions), coverage(recs, all_items)

print("{:<30} {:<15} {:<15}".format("Model", "Precision@10", "Coverage"))
for name, model in models.items():
    p, c = evaluate(model, test_df)
    print("{:<30} {:<15.4f} {:<15.4f}".format(name, p, c))

## 4. Explainability

In [ ]:
explainer = Explainer(loader, train_df)
user_id = test_df['userId'].iloc[0]
recs = models['Diversity Optimized (MMR)'].recommend(user_id, n=3)

print(f"User {user_id} Recommendations (Diversity Optimized):")
for mid in recs:
    title = loader.movies[loader.movies['movieId'] == mid]['title'].iloc[0]
    reason = explainer.explain(user_id, mid)
    print(f"- {title}\n  {reason}")